In [68]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

In [76]:
//val base_date = java.sql.Timestamp.valueOf("2017-01-01 00:00:00.0")
val base_date = "2017-01-01"

def get_months_between(from: Timestamp): Int = {
    val monthsBetween = ChronoUnit.MONTHS.between(
        from.toLocalDateTime().toLocalDate().withDayOfMonth(1),
        LocalDate.parse(base_date).withDayOfMonth(1)
    )
    monthsBetween.toInt
}
val udf_months_between = udf((t:Timestamp) => get_months_between(t))

base_date = 2017-01-01
udf_months_between = UserDefinedFunction(<function1>,IntegerType,Some(List(TimestampType)))


get_months_between: (from: java.sql.Timestamp)Int


UserDefinedFunction(<function1>,IntegerType,Some(List(TimestampType)))

# FlightData (passenger movements)

In [85]:
val moves = spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("dateFormat", "yyyy-MM-dd")
    .option("inferSchema", "true")
    .load("../resources/flightData.csv")
    .withColumn(
        "month", udf_months_between(col("date"))
    )
    .select("passengerId", "flightId", "month")
   .persist()
 
moves.printSchema()

root
 |-- passengerId: integer (nullable = true)
 |-- flightId: integer (nullable = true)
 |-- month: integer (nullable = false)



moves = [passengerId: int, flightId: int ... 1 more field]


[passengerId: int, flightId: int ... 1 more field]

In [86]:
moves.show(5)

+-----------+--------+-----+
|passengerId|flightId|month|
+-----------+--------+-----+
|         48|       0|    0|
|         94|       0|    0|
|         82|       0|    0|
|         21|       0|    0|
|         51|       0|    0|
+-----------+--------+-----+
only showing top 5 rows



In [7]:
moves.createOrReplaceTempView("moves")

# Flights
A flight is identified with a flightId, not the movement of each passenger, which is each row of the flightData.csv.

In [87]:
/*
val queryFlights = """
SELECT DISTINCT
    CAST(months_between(date, '%s') AS int) as month,
    flightId
FROM 
    moves 
ORDER BY 
    month, flightId
"""
.format(base_date)

val flights = spark.sql(queryFlights)
flights.show(5)
*/
/*
val queryNumFlightsPerMonth = """
SELECT DISTINCT
    month,
    COUNT(flightId)
FROM 
    flights
GROUP BY
    month
ORDER BY 
    month
"""

val numFlightsPerMonth = spark.sql(queryNumFlightsPerMonth)
*/

Name: Syntax Error.
Message: 
StackTrace: 

In [ ]:
val monthlyFlights = moves.pivot()

## Flights per month

In [9]:
val monthlyFlights = flights
    .groupBy("month")
    .count()
    .sort(asc("month"))
    .withColumnRenamed("count", "Flights")

monthlyFlights
    .coalesce(1)
    .write
    .format("csv")
    .mode(SaveMode.Overwrite)
    .option("header", "true")
    .csv("monthlyFlights.csv")

monthlyFlights = [month: int, Flights: bigint]


[month: int, Flights: bigint]

In [10]:
monthlyFlights.show(5)

+-----+-------+
|month|Flights|
+-----+-------+
|    0|     97|
|    1|     73|
|    2|     82|
|    3|     92|
|    4|     92|
+-----+-------+
only showing top 5 rows



In [13]:
flights.createOrReplaceTempView("flights")

lastException: Throwable = null


In [14]:
val queryNumFlightsPerMonth = """
SELECT DISTINCT
    month,
    COUNT(flightId)
FROM 
    flights
GROUP BY
    month
ORDER BY 
    month
"""

val numFlightsPerMonth = spark.sql(queryNumFlightsPerMonth)

queryNumFlightsPerMonth = 
numFlightsPerMonth = [month: int, count(flightId): bigint]


"
SELECT DISTINCT
    month,
    COUNT(flightId)
FROM
    flights
GROUP BY
    month
ORDER BY
    month
"


[month: int, count(flightId): bigint]

In [15]:
numFlightsPerMonth.show(5)

+-----+---------------+
|month|count(flightId)|
+-----+---------------+
|    0|             97|
|    1|             73|
|    2|             82|
|    3|             92|
|    4|             92|
+-----+---------------+
only showing top 5 rows



In [ ]:
org.apache.spark.sql.functions.months_between("")